<a href="https://colab.research.google.com/github/hmdjustina/Previsao-Jogos-da-Copa-2022/blob/main/Previs%C3%A3o_Jogos_da_Copa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
import random
from scipy.stats import poisson

selecoes = pd.read_excel('DadosCopaDoMundoQatar2022.xlsx', sheet_name ='selecoes', index_col = 0)
jogos = pd.read_excel('DadosCopaDoMundoQatar2022.xlsx', sheet_name ='jogos')

selecoes.head()

,Grupo,GrupoNumero,NomeEmIngles,PosiçãoRankingFIFA,ValorDeMercado,PontosRankingFIFA,Confederação,Copas,JogadorDestaque,FotoJogadorDestaque,LinkBandeiraPequena,LinkBandeiraGrande
Seleção,,,,,,,,,,,,
Catar,A,A1,Qatar,48,18.33,1442,AFC,0,Almoez Ali,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Equador,A,A2,Ecuador,44,125.80,1464,UEFA,0,Pervis Estupiñán,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Senegal,A,A3,Senegal,18,286.50,1585,CAF,0,Sadio Mané,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Holanda,A,A4,Netherlands,8,455.75,1679,UEFA,0,Virgil Van Dijk,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Inglaterra,B,B1,England,5,1360.00,1737,UEFA,1,Harry Kane,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...


In [28]:
jogos.head()

,data,hora,grupo,seleção1,seleção2,estádio
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR


In [29]:
fifa = selecoes['PontosRankingFIFA']
fifa

Seleção
Catar             1442
Equador           1464
Senegal           1585
Holanda           1679
Inglaterra        1737
Irã               1559
Estados Unidos    1635
País de Gales     1582
Argentina         1771
Arábia Saudita    1436
México            1650
Polônia           1546
França            1765
Dinamarca         1665
Tunísia           1508
Austrália         1484
Espanha           1717
Alemanha          1659
Japão             1555
Costa Rica        1500
Bélgica           1822
Canadá            1474
Marrocos          1558
Croácia           1632
Brasil            1838
Sérvia            1550
Suíça             1621
Camarões          1485
Portugal          1679
Gana              1393
Uruguai           1641
Coreia do Sul     1526
Name: PontosRankingFIFA, dtype: int64

In [30]:
a, b = min(fifa), max(fifa) 
fa, fb = 0.15, 1 
b1 = (fb - fa)/(b-a) 
b0 = fb - b*b1
forca = b0 + b1*fifa 

forca.sort_values(ascending = False)

Seleção
Brasil            1.000000
Bélgica           0.969438
Argentina         0.872022
França            0.860562
Inglaterra        0.807079
Espanha           0.768876
Holanda           0.696292
Portugal          0.696292
Dinamarca         0.669551
Alemanha          0.658090
México            0.640899
Uruguai           0.623708
Estados Unidos    0.612247
Croácia           0.606517
Suíça             0.585506
Senegal           0.516742
País de Gales     0.511011
Irã               0.467079
Marrocos          0.465169
Japão             0.459438
Sérvia            0.449888
Polônia           0.442247
Coreia do Sul     0.404045
Tunísia           0.369663
Costa Rica        0.354382
Camarões          0.325730
Austrália         0.323820
Canadá            0.304719
Equador           0.285618
Catar             0.243596
Arábia Saudita    0.232135
Gana              0.150000
Name: PontosRankingFIFA, dtype: float64

In [32]:
def Resultado(gols1, gols2):
    if gols1 > gols2:
        res = 'V'
    if gols1 < gols2:
        res = 'D' 
    if gols1 == gols2:
        res = 'E'       
    return res

def MediasPoisson(selecao1, selecao2):
    forca1 = forca[selecao1]
    forca2 = forca[selecao2] 
    mgols = 2.75
    l1 = mgols*forca1/(forca1 + forca2)
    l2 = mgols*forca2/(forca1 + forca2)
    return [l1, l2]
    
def Distribuicao(media):
    probs = []
    for i in range(7):
        probs.append(poisson.pmf(i,media))
    probs.append(1-sum(probs))
    return pd.Series(probs, index = ['0', '1', '2', '3', '4', '5', '6', '7+'])


def Pontos(gols1, gols2):
    rst = Resultado(gols1, gols2)
    if rst == 'V':
        pontos1, pontos2 = 3, 0
    if rst == 'E':
        pontos1, pontos2 = 1, 1
    if rst == 'D':
        pontos1, pontos2 = 0, 3
    return pontos1, pontos2, rst


def Jogo(selecao1, selecao2):
    l1, l2 = MediasPoisson(selecao1, selecao2)
    gols1 = int(np.random.poisson(lam = l1, size = 1))
    gols2 = int(np.random.poisson(lam = l2, size = 1))
    saldo1 = gols1 - gols2
    saldo2 = -saldo1
    pontos1, pontos2, result = Pontos(gols1, gols2)
    placar = '{}x{}'.format(gols1, gols2)
    return [gols1, gols2, saldo1, saldo2, pontos1, pontos2, result, placar]


def ProbabilidadesPartida(selecao1, selecao2):
    l1, l2 = MediasPoisson(selecao1, selecao2)
    d1, d2 = Distribuicao(l1), Distribuicao(l2)  
    matriz = np.outer(d1, d2)    #   Monta a matriz de probabilidades

    vitoria = np.tril(matriz).sum()-np.trace(matriz)    #Soma a triangulo inferior
    derrota = np.triu(matriz).sum()-np.trace(matriz)    #Soma a triangulo superior
    empate = 1 - (vitoria + derrota)
    probs = np.around([vitoria, empate , derrota], 3)
    probsp = [f'{100*i:.1f}%' for i in probs]

    nomes = ['0', '1', '2', '3', '4', '5', '6', '7+']
    matriz = pd.DataFrame(matriz, columns = nomes, index = nomes)
    matriz.index = pd.MultiIndex.from_product([[selecao1], matriz.index])
    matriz.columns = pd.MultiIndex.from_product([[selecao2], matriz.columns]) 

    output = {'seleção1': selecao1, 'seleção2': selecao2, 
             'f1': forca[selecao1], 'f2': forca[selecao2], 
             'media1': l1, 'media2': l2, 
             'probabilidades': probsp, 'matriz': matriz}
    return output

In [33]:
Jogo('Brasil', 'França')

[3, 1, 2, -2, 3, 0, 'V', '3x1']

In [34]:
ProbabilidadesPartida('Brasil', 'França')


{'seleção1': 'Brasil',
 'seleção2': 'França',
 'f1': 1.0,
 'f2': 0.8605617977528088,
 'media1': 1.478048191315901,
 'media2': 1.2719518086840993,
 'probabilidades': ['42.0%', '25.4%', '32.6%'],
 'matriz':              França                                                    \
                   0         1         2         3         4         5   
 Brasil 0   0.063928  0.081313  0.051713  0.021926  0.006972  0.001774   
        1   0.094488  0.120185  0.076435  0.032407  0.010305  0.002622   
        2   0.069829  0.088819  0.056487  0.023950  0.007616  0.001937   
        3   0.034404  0.043760  0.027830  0.011800  0.003752  0.000955   
        4   0.012713  0.016170  0.010284  0.004360  0.001386  0.000353   
        5   0.003758  0.004780  0.003040  0.001289  0.000410  0.000104   
        6   0.000926  0.001177  0.000749  0.000318  0.000101  0.000026   
        7+  0.000239  0.000303  0.000193  0.000082  0.000026  0.000007   
 
                                    
                 


Tarefa 1: **Prever os Jogos da Copa!**

In [35]:
jogos['Vitória'] = None
jogos['Empate'] = None
jogos['Derrota'] = None

for i in range(jogos.shape[0]):
    selecao1 = jogos['seleção1'][i]
    selecao2 = jogos['seleção2'][i]
    v, e, d = ProbabilidadesPartida(selecao1, selecao2)['probabilidades']
    jogos.at[i,'Vitória'] = v
    jogos.at[i,'Empate'] = e
    jogos.at[i,'Derrota'] = d

jogos.to_excel('outputEstimativasJogosCopa.xlsx')
jogos

,data,hora,grupo,seleção1,seleção2,estádio,Vitória,Empate,Derrota
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA,28.3%,24.9%,46.8%
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR,32.3%,25.4%,42.3%
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA,17.2%,22.1%,60.6%
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA,64.4%,21.0%,14.6%
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR,68.4%,19.6%,12.0%
5,2022-11-29,12:00:00,A,Equador,Senegal,INTER. KHALIFA,20.7%,23.3%,56.0%
6,2022-11-21,10:00:00,B,Inglaterra,Irã,INTER. KHALIFA,54.6%,23.6%,21.8%
7,2022-11-21,16:00:00,B,Estados Unidos,País de Gales,AL RAYYAN,43.0%,25.3%,31.7%
8,2022-11-25,07:00:00,B,País de Gales,Irã,AL RAYYAN,40.1%,25.5%,34.4%
9,2022-11-25,16:00:00,B,Inglaterra,Estados Unidos,AL KHOR,46.1%,25.0%,28.9%


Tarefa 2: **Simular a Copa do Mundo Inteira!**

In [36]:
def JogosGrupo(dados, grupo): 
    
    times = list(dados.loc[dados['Grupo']==grupo].index)

    time1, time2, time3, time4 = times  # nome dos times 

    pt1, pt2, pt3, pt4 = 0, 0, 0, 0
    gp1, gp2, gp3, gp4 = 0, 0, 0, 0
    sg1, sg2, sg3, sg4 = 0, 0, 0, 0

    jogo1 = Jogo(time1, time2)
    jogo2 = Jogo(time3, time4)

    jogo3 = Jogo(time1, time3)
    jogo4 = Jogo(time2, time4)

    jogo5 = Jogo(time1, time4)
    jogo6 = Jogo(time2, time3)

    gp1, gp2, sg1, sg2, pt1, pt2 = gp1 + jogo1[0], gp2 + jogo1[1], sg1 + jogo1[2], sg2 + jogo1[3], pt1 + jogo1[4], pt2 + jogo1[5]
    gp3, gp4, sg3, sg4, pt3, pt4 = gp3 + jogo2[0], gp4 + jogo2[1], sg3 + jogo2[2], sg4 + jogo2[3], pt3 + jogo2[4], pt4 + jogo2[5]
    gp1, gp3, sg1, sg3, pt1, pt3 = gp1 + jogo3[0], gp3 + jogo3[1], sg1 + jogo3[2], sg3 + jogo3[3], pt1 + jogo3[4], pt3 + jogo3[5]
    gp2, gp4, sg2, sg4, pt2, pt4 = gp2 + jogo4[0], gp4 + jogo4[1], sg2 + jogo4[2], sg4 + jogo4[3], pt2 + jogo4[4], pt4 + jogo4[5]
    gp1, gp4, sg1, sg4, pt1, pt4 = gp1 + jogo5[0], gp4 + jogo5[1], sg1 + jogo5[2], sg4 + jogo5[3], pt1 + jogo5[4], pt4 + jogo5[5]
    gp2, gp3, sg2, sg3, pt2, pt3 = gp2 + jogo6[0], gp3 + jogo6[1], sg2 + jogo6[2], sg3 + jogo6[3], pt2 + jogo6[4], pt3 + jogo6[5]

    partidas = [ time1 + ' x ' + time2, 
                 time3 + ' x ' + time4,
                 time1 + ' x ' + time3, 
                 time2 + ' x ' + time4,
                 time1 + ' x ' + time4,
                 time2 + ' x ' + time3 ]

    resultados = [ jogo1[6], jogo2[6], jogo3[6], jogo4[6], jogo5[6], jogo6[6] ]
    placares = [ jogo1[-1], jogo2[-1], jogo3[-1], jogo4[-1], jogo5[-1], jogo6[-1] ] 
    cols = ['Pontos', 'Saldo de Gols', 'Gols Pró']
    tab = pd.DataFrame([[pt1, pt2, pt3, pt4], [sg1, sg2, sg3, sg4], [gp1, gp2, gp3, gp4]], index = cols, columns = times).transpose()
    
    tab = tab.sort_values(['Pontos', 'Saldo de Gols', 'Gols Pró'], ascending = False)
    tab['Posição'] = [1, 2, 3, 4]

    jogos = pd.DataFrame([partidas, placares, resultados]).transpose()
    jogos.columns = ['Partida', 'Placar', 'Resultado']

    return [tab, jogos]

In [37]:
exemplo = JogosGrupo(selecoes, 'G')
exemplo

[          Pontos  Saldo de Gols  Gols Pró  Posição
 Sérvia         9              6         8        1
 Brasil         6              0         2        2
 Suíça          1             -3         4        3
 Camarões       1             -3         2        4,
              Partida Placar Resultado
 0    Brasil x Sérvia    0x2         D
 1   Suíça x Camarões    2x2         E
 2     Brasil x Suíça    1x0         V
 3  Sérvia x Camarões    2x0         V
 4  Brasil x Camarões    1x0         V
 5     Sérvia x Suíça    4x2         V]

In [38]:
exemplo[0]

,Pontos,Saldo de Gols,Gols Pró,Posição
Sérvia,9,6,8,1
Brasil,6,0,2,2
Suíça,1,-3,4,3
Camarões,1,-3,2,4


In [39]:
exemplo[1]

,Partida,Placar,Resultado
0,Brasil x Sérvia,0x2,D
1,Suíça x Camarões,2x2,E
2,Brasil x Suíça,1x0,V
3,Sérvia x Camarões,2x0,V
4,Brasil x Camarões,1x0,V
5,Sérvia x Suíça,4x2,V


Segunda Fase da Competição

In [40]:
def JogoMataMata(selecao1, selecao2):
  jogo = Jogo(selecao1, selecao2)
  resultado = jogo[6]
  if resultado == "V":
    return selecao1
  elif resultado == "D":
    return selecao2
  else:
    return random.sample([selecao1, selecao2], 1)[0]



In [41]:
def SimulaCopa(dados):
    cols = ['1st', '2nd', '3rd', '4th', 'Oitavas', 'Quartas', 'Semis', 'Final', 'Campeão']
    n = dados.shape[0]
    m = len(cols)
    aux = np.array(np.zeros(n*m).reshape(n, m))
    info = pd.DataFrame(aux, columns = cols, index = dados.index) 
    info = info.astype(int)

    top16 = []  
    for i in list('ABCDEFGH'):
        a = JogosGrupo(dados, i)[0] 
        top16 += a.index[:2].tolist()
        anomes = a.index.to_list() 
        info.at[anomes[0], '1st'] = 1
        info.at[anomes[1], '2nd'] = 1
        info.at[anomes[2], '3rd'] = 1
        info.at[anomes[3], '4th'] = 1

    qf1 = JogoMataMata(top16[0], top16[3])   #A1 x B2
    qf2 = JogoMataMata(top16[2], top16[1])   #B1 x A2
    qf3 = JogoMataMata(top16[4], top16[7])   #C1 x D2 
    qf4 = JogoMataMata(top16[6], top16[5])   #D1 x C2
    qf5 = JogoMataMata(top16[8], top16[11])  #E1 x F2
    qf6 = JogoMataMata(top16[10], top16[9])  #F1 x E2
    qf7 = JogoMataMata(top16[12], top16[15]) #G1 x H2
    qf8 = JogoMataMata(top16[14], top16[13]) #H1 x G2

    top8 = [qf1, qf2, qf3, qf4, qf5, qf6, qf7, qf8]


    sf1 = JogoMataMata(qf1, qf3)
    sf2 = JogoMataMata(qf2, qf4) 
    sf3 = JogoMataMata(qf5, qf7) 
    sf4 = JogoMataMata(qf6, qf8) 

    top4 = [sf1, sf2, sf3, sf4]

    f1 = JogoMataMata(sf1, sf3) 
    f2 = JogoMataMata(sf2, sf4) 
    top2 = [f1, f2]

    top1 = JogoMataMata(f1, f2)

    info.at[top16, 'Oitavas'] = 1
    info.at[top8, 'Quartas'] = 1
    info.at[top4, 'Semis'] = 1
    info.at[top2, 'Final'] = 1
    info.at[top1, 'Campeão'] = 1

    return info

In [42]:
SimulaCopa(selecoes)

,1st,2nd,3rd,4th,Oitavas,Quartas,Semis,Final,Campeão
Seleção,,,,,,,,,
Catar,1,0,0,0,1,0,0,0,0
Equador,0,1,0,0,1,1,0,0,0
Senegal,0,0,0,1,0,0,0,0,0
Holanda,0,0,1,0,0,0,0,0,0
Inglaterra,0,1,0,0,1,1,1,0,0
Irã,0,0,0,1,0,0,0,0,0
Estados Unidos,1,0,0,0,1,0,0,0,0
País de Gales,0,0,1,0,0,0,0,0,0
Argentina,0,0,1,0,0,0,0,0,0


In [43]:
SimulaCopa(selecoes).sum()

1st         8
2nd         8
3rd         8
4th         8
Oitavas    16
Quartas     8
Semis       4
Final       2
Campeão     1
dtype: int64

In [44]:
def SimulacaoTotal(dados, S = 1000): 
    print('IA: "Iniciando simulação..."')
    info = SimulaCopa(dados)
    for i in range(S-1):
        info += SimulaCopa(dados)
        if (i+2)%(S/10) == 0:
            print('IA: "Simulações de Copas do Mundo: {:.0f}% completas'.format(100*((i+2)/S)))    
    print('IA: "Simulação Finalizada!"')
    return info.sort_values(by = 'Campeão', ascending = False)/S

In [45]:
S = 10000
sim = SimulacaoTotal(selecoes, S) 
sim.to_excel('outputSimulaçõesCopaDoMundo.xlsx')
sim

IA: "Iniciando simulação..."
IA: "Simulações de Copas do Mundo: 10% completas
IA: "Simulações de Copas do Mundo: 20% completas
IA: "Simulações de Copas do Mundo: 30% completas
IA: "Simulações de Copas do Mundo: 40% completas
IA: "Simulações de Copas do Mundo: 50% completas
IA: "Simulações de Copas do Mundo: 60% completas
IA: "Simulações de Copas do Mundo: 70% completas
IA: "Simulações de Copas do Mundo: 80% completas
IA: "Simulações de Copas do Mundo: 90% completas
IA: "Simulações de Copas do Mundo: 100% completas
IA: "Simulação Finalizada!"


,1st,2nd,3rd,4th,Oitavas,Quartas,Semis,Final,Campeão
Seleção,,,,,,,,,
Brasil,0.5984,0.2527,0.1071,0.0418,0.8511,0.5663,0.3549,0.2207,0.1342
Bélgica,0.5797,0.2565,0.1203,0.0435,0.8362,0.5433,0.3357,0.2064,0.1238
Argentina,0.5389,0.2835,0.1392,0.0384,0.8224,0.4947,0.3064,0.1659,0.0939
França,0.5169,0.2970,0.1339,0.0522,0.8139,0.4828,0.2956,0.1653,0.0916
Inglaterra,0.4402,0.2701,0.1793,0.1104,0.7103,0.4550,0.2480,0.1354,0.0709
Espanha,0.4477,0.2860,0.1745,0.0918,0.7337,0.4017,0.2194,0.1173,0.0588
Holanda,0.5393,0.2845,0.1246,0.0516,0.8238,0.4476,0.2240,0.1085,0.0528
Portugal,0.4650,0.3272,0.1729,0.0349,0.7922,0.4200,0.2114,0.1043,0.0494
Dinamarca,0.3271,0.3632,0.2037,0.1060,0.6903,0.3547,0.1880,0.0929,0.0433


In [46]:
etapas = pd.DataFrame()
etapas['Cair na 1ª Fase'] = 1 - sim['Oitavas']
etapas['Cair nas Oitavas'] = sim['Oitavas'] - sim['Quartas']
etapas['Cair nas Quartas'] = sim['Quartas'] - sim['Semis']
etapas['Cair nas Semis'] = sim['Semis'] - sim['Final']
etapas['Cair na Final'] = sim['Final']  -  sim['Campeão'] 
etapas['Ganhar a Final'] = sim['Campeão']  
etapas.to_excel('outputProbabilidadesPorEtapa.xlsx')
etapas

,Cair na 1ª Fase,Cair nas Oitavas,Cair nas Quartas,Cair nas Semis,Cair na Final,Ganhar a Final
Seleção,,,,,,
Brasil,0.1489,0.2848,0.2114,0.1342,0.0865,0.1342
Bélgica,0.1638,0.2929,0.2076,0.1293,0.0826,0.1238
Argentina,0.1776,0.3277,0.1883,0.1405,0.0720,0.0939
França,0.1861,0.3311,0.1872,0.1303,0.0737,0.0916
Inglaterra,0.2897,0.2553,0.2070,0.1126,0.0645,0.0709
Espanha,0.2663,0.3320,0.1823,0.1021,0.0585,0.0588
Holanda,0.1762,0.3762,0.2236,0.1155,0.0557,0.0528
Portugal,0.2078,0.3722,0.2086,0.1071,0.0549,0.0494
Dinamarca,0.3097,0.3356,0.1667,0.0951,0.0496,0.0433


In [47]:
avanco = pd.DataFrame()
avanco['Avançar na 1ª Fase'] = sim['Oitavas']
avanco['Avançar nas Oitavas'] = sim['Quartas']/sim['Oitavas'] 
avanco['Avançar nas Quartas'] = sim['Semis']/sim['Quartas'] 
avanco['Avançar nas Semis'] = sim['Final']/sim['Semis'] 
avanco['Avançar na Final'] = sim['Campeão']/sim['Final']   
avanco.to_excel('outputAvançoPorEtapa.xlsx')
avanco

,Avançar na 1ª Fase,Avançar nas Oitavas,Avançar nas Quartas,Avançar nas Semis,Avançar na Final
Seleção,,,,,
Brasil,0.8511,0.665374,0.626700,0.621865,0.608065
Bélgica,0.8362,0.649725,0.617891,0.614835,0.599806
Argentina,0.8224,0.601532,0.619365,0.541449,0.566004
França,0.8139,0.593193,0.612262,0.559202,0.554144
Inglaterra,0.7103,0.640574,0.545055,0.545968,0.523634
Espanha,0.7337,0.547499,0.546179,0.534640,0.501279
Holanda,0.8238,0.543336,0.500447,0.484375,0.486636
Portugal,0.7922,0.530169,0.503333,0.493377,0.473634
Dinamarca,0.6903,0.513835,0.530025,0.494149,0.466093
